### 라이브러리

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [2]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [3]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [4]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [5]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [6]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [7]:
# month1 = {
#     1:3,
#     2:1,
#     3:2,
#     4:6,
#     5:7,
#     6:8,
#     7:10,
#     8:9,
#     9:5,
#     10:4,
#     11:11,
#     12:12
# }

month1 = {
    6:12,
	5:11,
	8:10,
	7:9	,
	10:8,
	11:7,
	9:6,
	4:5,
	2:4,
	3:3,
	1:2,
	12:1
}



month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [8]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [9]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

In [10]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [11]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

### TOP Score

In [12]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [13]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [14]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7657572	total: 143ms	remaining: 2m 22s
1:	learn: 199.0244251	total: 145ms	remaining: 1m 12s
2:	learn: 194.0322793	total: 148ms	remaining: 49s
3:	learn: 189.5959005	total: 149ms	remaining: 37.1s
4:	learn: 185.4566813	total: 151ms	remaining: 30s
5:	learn: 181.2524975	total: 153ms	remaining: 25.3s
6:	learn: 177.1912381	total: 155ms	remaining: 21.9s
7:	learn: 173.2319401	total: 156ms	remaining: 19.4s
8:	learn: 169.5132883	total: 159ms	remaining: 17.5s
9:	learn: 166.0075568	total: 161ms	remaining: 15.9s
10:	learn: 162.5962776	total: 163ms	remaining: 14.6s
11:	learn: 159.4383330	total: 164ms	remaining: 13.5s
12:	learn: 156.6007678	total: 166ms	remaining: 12.6s
13:	learn: 154.2442645	total: 168ms	remaining: 11.8s
14:	learn: 151.4360531	total: 170ms	remaining: 11.2s
15:	learn: 149.0920455	total: 172ms	remaining: 10.6s
16:	learn: 146.6930090	total: 174ms	remaining: 10s
17:	learn: 144.4868467	total: 176ms	remaining: 9.58s
18:	learn: 142.4751438	total: 

In [17]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1020.373486,166.642829
1,2021-01-28,936.881259,494.388267
2,2021-01-29,537.888905,183.386029


In [16]:
submission

,일자,중식계,석식계
0,2021-01-27,1020.373486,166.642829
1,2021-01-28,936.881259,494.388267
2,2021-01-29,537.888905,183.386029
3,2021-02-01,1231.633778,501.400934
4,2021-02-02,1046.988826,563.052477
5,2021-02-03,1022.872828,504.399528
6,2021-02-04,998.936576,564.049625
7,2021-02-05,742.158380,312.156024
8,2021-02-08,1231.041688,680.363388
9,2021-02-09,1059.599943,586.775224


In [19]:
submission.to_csv('ko4(랭크변경).csv', index=False)

In [52]:
test.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '요일(석식)', '년', '월', '일', '주', '식사가능자수', '월(중식)', '월(석식)',
       '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2', '공휴일전후2_0', '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

In [53]:
test[['일자','요일','식사가능자수','본사시간외근무명령서승인건수']]

,일자,요일,식사가능자수,본사시간외근무명령서승인건수
0,2021-01-27,2,2537.0,5
1,2021-01-28,3,2531.0,409
2,2021-01-29,4,2419.0,0
3,2021-02-01,0,2494.0,538
4,2021-02-02,1,2548.0,455
5,2021-02-03,2,2579.0,5
6,2021-02-04,3,2575.0,476
7,2021-02-05,4,2499.0,0
8,2021-02-08,0,2507.0,690
9,2021-02-09,1,2501.0,542


In [54]:
train.query('석식계<300')[['일자','요일','석식계','본사시간외근무명령서승인건수','식사가능자수']]

,일자,요일,석식계,본사시간외근무명령서승인건수,식사가능자수
152,2016-09-13,1,238.0,52,2258.0
204,2016-11-30,2,0.0,0,2621.0
220,2016-12-22,3,213.0,32,2587.0
224,2016-12-28,2,0.0,0,2539.0
226,2016-12-30,4,275.0,3,2176.0
244,2017-01-25,2,0.0,0,2618.0
245,2017-01-26,3,147.0,28,2328.0
262,2017-02-22,2,0.0,0,2557.0
281,2017-03-22,2,0.0,0,2574.0
306,2017-04-26,2,0.0,0,2581.0


In [78]:
train.query("년>=2020 & 요일==2 & 석식계>0")['석식계'].mean()

385.5531914893617

In [107]:
train.query("요일==2 & 석식계>0")['석식계'].mean()

428.1427935547048

In [25]:
train.query('석식계<10')['석식계']

204     0.0
224     0.0
244     0.0
262     0.0
281     0.0
306     0.0
327     0.0
346     0.0
366     0.0
392     0.0
410     0.0
412     0.0
424     0.0
449     0.0
468     0.0
492     0.0
502     0.0
510     0.0
529     0.0
549     0.0
571     0.0
589     0.0
609     0.0
633     0.0
648     0.0
667     0.0
687     0.0
706     0.0
730     0.0
747     0.0
766     0.0
786     0.0
809     0.0
828     0.0
853     0.0
872     0.0
890     0.0
912     0.0
932     0.0
955     0.0
973     0.0
993     0.0
1166    0.0
Name: 석식계, dtype: float64

In [29]:
train['석식계']

0       331.0
1       560.0
2       573.0
3       525.0
4       330.0
        ...  
1200    421.0
1201    353.0
1202    217.0
1203    502.0
1204    480.0
Name: 석식계, Length: 1205, dtype: float64

In [28]:
# train[train['석식계'] < 10 ]['석식계']

In [172]:
train

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,주,식사가능자수,월(중식),월(석식),주(중식),주(석식),공휴일전후1_0,공휴일전후1_1,공휴일전후1_2,공휴일전후2_0,공휴일전후2_1,공휴일전후2_2
0,2016-02-01 00:00:00,0,2601,50,150,238,0.0,1039.0,331.0,1,2016,2,1,5,2551.0,1,2,7,31,1,0,0,1,0,0
1,2016-02-02 00:00:00,1,2601,50,173,319,0.0,867.0,560.0,2,2016,2,2,5,2551.0,1,2,7,31,1,0,0,1,0,0
2,2016-02-03 00:00:00,2,2601,56,180,111,0.0,1017.0,573.0,5,2016,2,3,5,2545.0,1,2,7,31,1,0,0,1,0,0
3,2016-02-04 00:00:00,3,2601,104,220,355,0.0,978.0,525.0,3,2016,2,4,5,2497.0,1,2,7,31,1,0,0,1,0,0
4,2016-02-05 00:00:00,4,2601,278,181,34,0.0,925.0,330.0,4,2016,2,5,5,2323.0,1,2,7,31,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20 00:00:00,2,2983,75,198,4,391.0,1093.0,421.0,5,2021,1,20,3,2517.0,3,11,22,35,1,0,0,1,0,0
1201,2021-01-21 00:00:00,3,2983,92,231,462,351.0,832.0,353.0,3,2021,1,21,3,2540.0,3,11,22,35,1,0,0,1,0,0
1202,2021-01-22 00:00:00,4,2983,255,248,1,303.0,579.0,217.0,4,2021,1,22,3,2425.0,3,11,22,35,1,0,0,1,0,0
1203,2021-01-25 00:00:00,0,2983,107,153,616,327.0,1145.0,502.0,1,2021,1,25,4,2549.0,3,11,17,49,1,0,0,1,0,0


# 0719
자기개발의날을 2020년 수요일 평균돌리고 그렇게하면 금욜이 랭크상 젤 낮아져서 요일랭크 수정하고 돌림.
2/24. 3/31이 350 400에서 효과가 좋으니 따로 조금 낮추었음. 다만 이것들은 퍼블릭이라 제출땐 안해도됨.


In [27]:
train.groupby('요일')['중식계'].mean()

요일
0    1146.395833
1     925.620833
2     905.213389
3     823.991803
4     653.590909
Name: 중식계, dtype: float64

In [28]:
train.groupby('월')['중식계'].mean()

월
1     934.247525
2     989.623656
3     953.607477
4     883.621359
5     869.787879
6     863.311111
7     848.736364
8     854.785047
9     907.148936
10    917.771739
11    847.333333
12    827.336634
Name: 중식계, dtype: float64

In [31]:
train.query('중식계 < 400')

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,요일(석식),년,월,일,주,식사가능자수,월(중식),월(석식),공휴일전후1_0,공휴일전후1_1,공휴일전후1_2,공휴일전후2_0,공휴일전후2_1,공휴일전후2_2
310,2017-05-04,3,2637,1224,41,45,0.0,383.0,262.0,3,2017,5,4,18,1413.0,7,8,0,0,1,0,0,1
469,2017-12-28,3,2665,240,260,55,0.0,311.0,245.0,3,2017,12,28,52,2425.0,12,12,1,0,0,1,0,0
470,2017-12-29,4,2665,489,242,5,0.0,394.0,209.0,5,2017,12,29,52,2176.0,12,12,0,1,0,0,1,0
705,2018-12-24,0,2846,894,159,26,0.0,296.0,104.0,1,2018,12,24,52,1952.0,12,12,0,0,1,0,0,1
709,2018-12-31,0,2846,1085,132,28,0.0,332.0,135.0,1,2018,12,31,1,1761.0,12,12,0,0,1,0,0,1
815,2019-06-07,4,2758,890,219,3,0.0,389.0,244.0,5,2019,6,7,23,1868.0,8,6,0,0,1,0,0,1
953,2019-12-27,4,3111,706,236,1,0.0,387.0,226.0,5,2019,12,27,52,2405.0,12,12,0,0,1,0,0,1
955,2019-12-31,1,3111,709,149,22,0.0,349.0,0.0,2,2019,12,31,1,2402.0,12,12,0,1,0,0,1,0
1129,2020-09-28,0,3132,985,188,81,142.0,358.0,165.0,1,2020,9,28,40,2005.0,5,5,0,0,1,0,0,1
1148,2020-10-30,4,3008,340,298,1,107.0,398.0,259.0,5,2020,10,30,44,2561.0,4,3,1,0,0,1,0,0
